# Goal
In this project I'm going to build a predictor for sales that occur for shop with id = 5.

### Necessary imports

In [97]:
import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
import sklearn as sk

### Load data

In [98]:
df_train = pd.read_csv('data/sales_train.csv')
df_test = pd.read_csv('data/test.csv')
df_sample = pd.read_csv('data/sample_submission.csv')
df_shops = pd.read_csv('data/shops.csv')
df_items = pd.read_csv('data/items.csv')
df_item_categories = pd.read_csv('data/item_categories.csv')

In [99]:
print('Train sample\n', df_train.head(10))
print('Test sample\n', df_test.head(10))
print('Sample submission\n', df_sample.head(10))
print('Shops\n', df_shops.head(10))
print('Items\n', df_items.head(10))
print('Item categories\n', df_item_categories.head(10))

Train sample
          date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
5  10.01.2013               0       25     2564      349.00           1.0
6  02.01.2013               0       25     2565      549.00           1.0
7  04.01.2013               0       25     2572      239.00           1.0
8  11.01.2013               0       25     2572      299.00           1.0
9  03.01.2013               0       25     2573      299.00           3.0
Test sample
    ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
5   5        5 

### Outlier removal

In [100]:
df_train[df_train['item_cnt_day'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2,05.01.2013,0,25,2552,899.0,-1.0
148,23.01.2013,0,25,2321,999.0,-1.0
175,07.01.2013,0,25,2199,1449.0,-1.0
807,02.01.2013,0,25,2330,599.0,-1.0
1041,13.01.2013,0,25,5034,1989.0,-1.0
...,...,...,...,...,...,...
2934243,26.10.2015,33,25,3917,449.0,-1.0
2934462,18.10.2015,33,25,4896,6398.0,-1.0
2935263,05.10.2015,33,25,10039,249.0,-1.0
2935643,16.10.2015,33,25,7893,2990.0,-1.0


In [101]:
df_train.loc[df_train['item_cnt_day'] < 0, 'item_cnt_day'] = df_train['item_cnt_day'].mean().round()

In [102]:
print('Train sample\n', df_train.head(10))

Train sample
          date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00           1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
5  10.01.2013               0       25     2564      349.00           1.0
6  02.01.2013               0       25     2565      549.00           1.0
7  04.01.2013               0       25     2572      239.00           1.0
8  11.01.2013               0       25     2572      299.00           1.0
9  03.01.2013               0       25     2573      299.00           3.0


In [103]:
df_train.loc[df_train['item_price'] < 0, 'item_price'] = df_train['item_price'].mean()

In [104]:
df_train[df_train['item_price'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


In [105]:
df_train[df_train.isna().any(1)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


Sort data by date, but first convert it to datetime to make up for a fact that it's in european format

In [108]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_train = df_train.sort_values('date')
print(df_train.head(70000))

             date  date_block_num  shop_id  item_id  item_price  item_cnt_day
109593 2013-01-01               0       46    18616       349.0           1.0
107443 2013-01-01               0       37    22087        79.0           2.0
75872  2013-01-01               0       54    17111       149.0           1.0
85162  2013-01-01               0       54    11854       199.0           1.0
18128  2013-01-01               0       28     4906      1799.0           1.0
...           ...             ...      ...      ...         ...           ...
111675 2013-01-22               0       42       32       349.0           1.0
78849  2013-01-22               0       59    11220       118.6           1.0
53596  2013-01-22               0       10     5378      2266.0           1.0
82385  2013-01-22               0       54     3470       349.0           1.0
58642  2013-01-22               0       12    13450      2799.0           1.0

[70000 rows x 6 columns]


## Exploratory data analysis
Form a dataframe with date, revenue and sale count per day

In [109]:
date = None
sale_cnt = None
price = None
df = pd.DataFrame(columns = ['date', 'revenue', 'item_cnt_day'])
for index, row in df_train.iterrows():
    if row.at['date'] == date:
        sale_cnt += row.at['item_cnt_day']
        price += row.at['item_price']
    else:
        date = row.at['date']
        sale_cnt = row.at['item_cnt_day']
        price = row.at['item_price']
        df_row = pd.DataFrame({'date' : [date], 'revenue' : [sale_cnt*price], 'item_cnt_day' : [sale_cnt]})
        df = df.append(df_row)
    #Check for last row
    if index == len(df_train_shop) - 1:
        sale_cnt += row.at['item_cnt_day']
        price += row.at['item_price']
        df_row = pd.DataFrame({'date' : [date], 'revenue' : [sale_cnt*price], 'item_cnt_day' : [sale_cnt]})
        df = df.append(df_row)

In [110]:
print(df.head(30))

        date       revenue  item_cnt_day
0 2013-01-01  3.490000e+02           1.0
0 2013-01-02  7.600000e+01           1.0
0 2013-01-03  4.490000e+02           1.0
0 2013-01-04  1.490000e+02           1.0
0 2013-01-05  2.190000e+02           1.0
0 2013-01-06  1.999000e+03           1.0
0 2013-01-07  2.500000e+01           1.0
0 2013-01-08  6.690000e+02           5.0
0 2013-01-09  1.990000e+02           1.0
0 2013-01-10  4.990000e+02           1.0
0 2013-01-11  6.490000e+02           1.0
0 2013-01-12  3.990000e+02           1.0
0 2013-01-13  1.799000e+03           1.0
0 2013-01-14  5.990000e+02           1.0
0 2013-01-14  3.621006e+09        2290.0
0 2013-01-15  2.990000e+02           1.0
0 2013-01-16  1.549000e+03           1.0
0 2013-01-17  1.490000e+02           1.0
0 2013-01-18  9.890000e+02           1.0
0 2013-01-19  3.490000e+02           1.0
0 2013-01-20  1.990000e+02           1.0
0 2013-01-21  7.900000e+01           1.0
0 2013-01-22  2.490000e+02           1.0
0 2013-01-23  5.